In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os

In [2]:
topic_file = '/Users/bew/Downloads/DataForThai_Pandas&SubTopic/SubTopic_dataforthai.csv'
topic_table = pd.read_csv(topic_file)


topic_table['รหัส'] = topic_table['รหัส'].astype(str)
# Function to add '0' to strings of length 4
def add_zero(s):
    if len(s) == 4:
        return '0' + s
    else:
        return s
# Applying the function to the DataFrame
topic_table['รหัส'] = topic_table['รหัส'].apply(lambda x: add_zero(x))


# topic_table.dtypes
topic_table = topic_table[608:628]
# topic_table = topic_table[topic_table['รหัส']=='46311']
# print(topic_table)
# max(topic_table['จำนวนหน้า'])
topic_table

,รหัส,ประเภทธุรกิจ,จำนวน,จำนวนหน้า
608,46101,การขายส่งวัตถุดิบทางการเกษตรและสัตว์มีชีวิตโดย...,1177,3
609,46102,การขายส่งอาหารเครื่องดื่มและยาสูบโดยได้รับค่าต...,1933,5
610,46103,การขายส่งสิ่งทอเสื้อผ้ารองเท้าเครื่องหนังและขอ...,2544,7
611,46104,การขายส่งคอมพิวเตอร์และซอฟต์แวร์อุปกรณ์อิเล็กท...,1004,3
612,46105,การขายส่งเครื่องจักรอุตสาหกรรมโดยได้รับค่าตอบแ...,984,3
613,46106,การขายส่งเชื้อเพลิงสินแร่โลหะโดยได้รับค่าตอบแท...,292,1
614,46107,การขายส่งวัสดุที่ใช้ในการก่อสร้างผลิตภัณฑ์โลหะ...,952,3
615,46108,การขายส่งสินค้าเฉพาะอย่างอื่นๆซึ่งมิได้จัดประเ...,734,2
616,46109,การขายส่งสินค้าทั่วไปโดยได้รับค่าตอบแทนหรือตาม...,7485,19
617,46200,การขายส่งวัตถุดิบทางการเกษตรและสัตว์มีชีวิต,1,1


In [3]:
len(topic_table)

20

In [4]:
# รวมไฟล์

# Initialize the result DataFrame
result = pd.DataFrame()

# Iterate through each row in topic_table
for index, row in topic_table.iterrows():
    
    value1 = row['รหัส']
    
    for i in range(1,150):
        i_str = str(i)  # Convert i to string
        file = f'/Users/bew/Downloads/DataForThai_Pandas&SubTopic/pf2/{value1}_{i_str}.html'

        # Read the HTML file
        try:
            # Read the HTML file
            with open(file, 'r', encoding='utf-8') as file:
                html_content = file.read()
        except FileNotFoundError:
#             check file 1 ว่ามี 400 rows ไหม เผื่อพลาดข้อมูลหน้า 2+
            if i == 2:
                check_file_400 = f'/Users/bew/Downloads/DataForThai_Pandas&SubTopic/pf2/{value1}_1.html'
                check_df_file_1 = pd.read_html(check_file_400)
                check_df_file_1 = check_df_file_1[1]
                if len(check_df_file_1) == 400:
                    print(f"{value1} have full 1 page (check for maybe have more pages)")
            # If the file is not found, break the loop
            break

        # Parse the HTML content
        soup = BeautifulSoup(html_content, 'lxml')

        # Find the table (modify as needed to target the correct table)
        table = soup.find('table', {'class': 'datatable'})

        # Initialize an empty list to store the rows
        data = []

        # Iterate over each row in the table
        for row in table.find_all('tr', class_='click-able-row'):
            # Extract the text of each cell
            cells = row.find_all('td')
            row_data = [cell.text.strip() for cell in cells]

            # Extract the onclick attribute
            onclick = row.get('onclick')
            if onclick:
                # Extract the desired values from the onclick string
                parts = onclick.split("'")
                if len(parts) > 1:
                    company_id = parts[1]
                    company_name = parts[3]
                    row_data.append(company_id)
                    row_data.append(company_name)

            data.append(row_data)

        # Convert to DataFrame
        columns = ['Order', 'Name', 'Hidden1', 'Capital', 'Hidden2', 'Province', 'District', 'Company ID', 'Company Name']
        df = pd.DataFrame(data, columns=columns)
        
        df['Capital'] = df['Capital'].str.replace(',', '')
        df['Capital'] = pd.to_numeric(df['Capital'])
        df = df[(df['Capital'] >= 30000000)]
        df = df.reset_index(drop=True)
        
        df = df.drop(columns=['Order', 'Name', 'Hidden1', 'Capital', 'Hidden2', 'Province', 'District', 'Company Name'])

        # Concatenate df and result DataFrames
        result = pd.concat([result, df], axis=0, ignore_index=True)
        
        
#     # Specify the file name and file path
#     file_name = value1+'.csv'
#     file_path = '/Users/bew/Downloads/WebScrapperFor_DataForThai/DataForThaiCompanyIdsByCategories/'

#     # Combine the file path and file name
#     full_path = os.path.join(file_path, file_name)

#     result.to_csv(full_path, sep=',', index=False, header=True, encoding='utf-8')
    print(f"{value1} Complete")
#     df_null = []
#     result = pd.DataFrame(df_null, columns=columns)
#     result = result.drop(columns=['Order', 'Name', 'Hidden1', 'Capital', 'Hidden2', 'Province', 'District', 'Company Name'])

    # Reset the index of the final result DataFrame
#     result = result.reset_index(drop=True)

# result

46101 Complete
46102 Complete
46103 Complete
46104 Complete
46105 Complete
46106 Complete
46107 Complete
46108 Complete
46109 Complete
46200 Complete
46201 Complete
46202 Complete
46203 Complete
46204 Complete
46205 Complete
46206 Complete
46209 Complete
46300 Complete
46310 Complete
46311 Complete


In [5]:
result = result.reset_index(drop=True)

result

,Company ID
0,0105557044676
1,0735551004764
2,0225569000930
3,0105551056509
4,0225563001630
...,...
591,0105552104292
592,0105532041902
593,0105545112863
594,0107552000297


In [6]:
len(result)                                                     # ข้อมูลบริษัททั้งหมด = 44,318 (แต่ยังไม่กรองการดำเนินธุรกิจ)

596

In [7]:
# # แยกไฟล์ตามหัวข้อ

# # Initialize the result DataFrame
# result = pd.DataFrame()

# # Iterate through each row in topic_table
# for index, row in topic_table.iterrows():
    
#     value1 = row['รหัส']
    
#     for i in range(1,150):
#         i_str = str(i)  # Convert i to string
#         file = f'/Users/bew/Downloads/DataForThai_Pandas&SubTopic/pf2/{value1}_{i_str}.html'

#         # Read the HTML file
#         try:
#             # Read the HTML file
#             with open(file, 'r', encoding='utf-8') as file:
#                 html_content = file.read()
#         except FileNotFoundError:
# #             check file 1 ว่ามี 400 rows ไหม เผื่อพลาดข้อมูลหน้า 2+
#             if i == 2:
#                 check_file_400 = f'/Users/bew/Downloads/DataForThai_Pandas&SubTopic/pf2/{value1}_1.html'
#                 check_df_file_1 = pd.read_html(check_file_400)
#                 check_df_file_1 = check_df_file_1[1]
#                 if len(check_df_file_1) == 400:
#                     print(f"{value1} have full 1 page (check for maybe have more pages)")
#             # If the file is not found, break the loop
#             break

#         # Parse the HTML content
#         soup = BeautifulSoup(html_content, 'lxml')

#         # Find the table (modify as needed to target the correct table)
#         table = soup.find('table', {'class': 'datatable'})

#         # Initialize an empty list to store the rows
#         data = []

#         # Iterate over each row in the table
#         for row in table.find_all('tr', class_='click-able-row'):
#             # Extract the text of each cell
#             cells = row.find_all('td')
#             row_data = [cell.text.strip() for cell in cells]

#             # Extract the onclick attribute
#             onclick = row.get('onclick')
#             if onclick:
#                 # Extract the desired values from the onclick string
#                 parts = onclick.split("'")
#                 if len(parts) > 1:
#                     company_id = parts[1]
#                     company_name = parts[3]
#                     row_data.append(company_id)
#                     row_data.append(company_name)

#             data.append(row_data)

#         # Convert to DataFrame
#         columns = ['Order', 'Name', 'Hidden1', 'Capital', 'Hidden2', 'Province', 'District', 'Company ID', 'Company Name']
#         df = pd.DataFrame(data, columns=columns)
        
#         df['Capital'] = df['Capital'].str.replace(',', '')
#         df['Capital'] = pd.to_numeric(df['Capital'])
#         df = df[(df['Capital'] >= 30000000)]
#         df = df.reset_index(drop=True)
        
#         df = df.drop(columns=['Order', 'Name', 'Hidden1', 'Capital', 'Hidden2', 'Province', 'District', 'Company Name'])

#         # Concatenate df and result DataFrames
#         result = pd.concat([result, df], axis=0, ignore_index=True)
        
        
#     # Specify the file name and file path
#     file_name = value1+'.csv'
#     file_path = '/Users/bew/Downloads/WebScrapperFor_DataForThai/DataForThaiCompanyIdsByCategories/'

#     # Combine the file path and file name
#     full_path = os.path.join(file_path, file_name)

#     result.to_csv(full_path, sep=',', index=False, header=True, encoding='utf-8')
#     print(f"{value1} Complete")
#     df_null = []
#     result = pd.DataFrame(df_null, columns=columns)
#     result = result.drop(columns=['Order', 'Name', 'Hidden1', 'Capital', 'Hidden2', 'Province', 'District', 'Company Name'])

#     # Reset the index of the final result DataFrame
#     result = result.reset_index(drop=True)

# result

In [8]:
print("finish")

finish


In [9]:
# value1 = '01111'

# file = '/Users/bew/Downloads/GetData_DataForThai/passthrough/'+value1+'_1.html'
# # Read the HTML file
# with open(file, 'r', encoding='utf-8') as file:
#     html_content = file.read()

# # Parse the HTML content
# soup = BeautifulSoup(html_content, 'lxml')

# # Find the table (modify as needed to target the correct table)
# table = soup.find('table', {'class': 'datatable'})

# # # Convert the table to a pandas DataFrame
# # df = pd.read_html(str(table))[0]

# # Initialize an empty list to store the rows
# data = []

# # Iterate over each row in the table
# for row in table.find_all('tr', class_='click-able-row'):
#     # Extract the text of each cell
#     cells = row.find_all('td')
#     row_data = [cell.text.strip() for cell in cells]

#     # Extract the onclick attribute
#     onclick = row.get('onclick')
#     if onclick:
#         # Extract the desired values from the onclick string
#         parts = onclick.split("'")
#         if len(parts) > 1:
#             company_id = parts[1]
#             company_name = parts[3]
#             row_data.append(company_id)
#             row_data.append(company_name)

#     data.append(row_data)

# # Convert to DataFrame
# columns = ['Order', 'Name', 'Hidden1', 'Capital', 'Hidden2', 'Province', 'District', 'Company ID', 'Company Name']
# df = pd.DataFrame(data, columns=columns)
# # df = df.drop(columns=['Order', 'Name', 'Hidden1', 'Hidden2', 'Province', 'District'])
# df = df.drop(columns=['Order', 'Name', 'Hidden1', 'Hidden2', 'Province', 'District'])

# df

In [10]:
# value2 = '01112'

# file2 = '/Users/bew/Downloads/GetData_DataForThai/passthrough/'+value2+'_1.html'
# # Read the HTML file
# with open(file2, 'r', encoding='utf-8') as file:
#     html_content = file.read()

# # Parse the HTML content
# soup = BeautifulSoup(html_content, 'lxml')

# # Find the table (modify as needed to target the correct table)
# table = soup.find('table', {'class': 'datatable'})

# # # Convert the table to a pandas DataFrame
# # df = pd.read_html(str(table))[0]

# # Initialize an empty list to store the rows
# data = []

# # Iterate over each row in the table
# for row in table.find_all('tr', class_='click-able-row'):
#     # Extract the text of each cell
#     cells = row.find_all('td')
#     row_data = [cell.text.strip() for cell in cells]

#     # Extract the onclick attribute
#     onclick = row.get('onclick')
#     if onclick:
#         # Extract the desired values from the onclick string
#         parts = onclick.split("'")
#         if len(parts) > 1:
#             company_id = parts[1]
#             company_name = parts[3]
#             row_data.append(company_id)
#             row_data.append(company_name)

#     data.append(row_data)

# # Convert to DataFrame
# columns = ['Order', 'Name', 'Hidden1', 'Capital', 'Hidden2', 'Province', 'District', 'Company ID', 'Company Name']
# df2 = pd.DataFrame(data, columns=columns)
# # df = df.drop(columns=['Order', 'Name', 'Hidden1', 'Hidden2', 'Province', 'District'])
# df2 = df2.drop(columns=['Order', 'Name', 'Hidden1', 'Hidden2', 'Province', 'District'])

# df2

In [11]:
# file = '/Users/bew/Downloads/GetData_DataForThai/passthrough/01111_1.html'
# df = pd.read_html(file)
# df = df[1]
# # df.dtypes
# df

In [12]:
# i = '1'
# file = '/Users/bew/Downloads/GetData_DataForThai/passthrough/01112_1.html'
# df = pd.read_html(file)
# df = df[1]
# df

In [13]:
# # for index, row in df.iterrows():
# file_m = '/Users/bew/Downloads/GetData_DataForThai/passthrough/01112_1.html'
# df_m = pd.read_html(file_m)
# df_m = df_m[1]
# df_m

In [14]:
# result = pd.concat([df, df_m], axis=0)
# # print(result)

# result = result.reset_index(drop=True)
# result